## People Centered COS

### Attempt to map string location to Lat Long

#### Analysis Questions
  
- Can we map the string location 1 and location 2 to latitude and longitude values reliably?

### Set Up 

In [1]:
pip install geocoder


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install plotly


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import geocoder

from geopy.geocoders import Nominatim

In [7]:
pd.set_option('display.max_rows', 500)

### Load CDOT Data from 2020 - 2024

In [9]:
import sys  
sys.path.insert(1, '../../')

from convert_old_cdot_format_to_new_format import convert_old_cdot_format_to_new_format

In [44]:
# Load CDOT Data for a list of years

years = [2020, 2021, 2022, 2023, 2024]

cdot_base_path = "/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/"

loaded_data_list = []
for y in years:
    filename = f"CDOTRM_CD_Crash_Listing_-_{y}.xlsx"
    full_path = cdot_base_path + filename
    print(full_path)
    year_cdot_pdf = pd.read_excel(cdot_base_path + filename)
    year_cdot_pdf['Crash Year'] = y
    print(list(year_cdot_pdf.columns)[0:5])
    print(year_cdot_pdf.shape)
    
    if y < 2021:
        # call conversion code 
        year_cdot_pdf = convert_old_cdot_format_to_new_format(year_cdot_pdf)

    loaded_data_list.append(year_cdot_pdf) 
    
cdot_pdf_load = pd.concat(loaded_data_list)

/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2020.xlsx
['SYSTEM', 'RTE', 'SEC', 'MP', 'DATE']
(86304, 85)


/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/cdot_geography_cleaning/../../convert_old_cdot_format_to_new_format.py:403: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = ''
/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/cdot_geography_cleaning/../../convert_old_cdot_format_to_new_format.py:403: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = ''
/Users/haley.northrup/Desktop/PeopleCenteredCOS/beautiful_cos_data_analysis/sandbox/cd

/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2021.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(97848, 111)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2022.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(95431, 111)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2023.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(102166, 111)
/Users/haley.northrup/Desktop/PeopleCenteredCOS/cdot_data/CDOTRM_CD_Crash_Listing_-_2024.xlsx
['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street']
(99980, 111)


### Clean Data

In [45]:
print('Loaded Data Size', cdot_pdf_load.shape)

# Drop duplicates
cdot_pdf_withNans_dedup = cdot_pdf_load.drop_duplicates()
print('Data size after drop duplicates', cdot_pdf_withNans_dedup.shape)

display(cdot_pdf_withNans_dedup.head(5))

Loaded Data Size (481729, 149)
Data size after drop duplicates (481615, 149)


,System Code,Crash Date,Injury 00,Injury 01,Injury 02,Injury 03,Injury 04,Agency Id,Location,Road Description,...,Wild Animal,Road Contour Curves,Road Contour Grade,Number Killed,Number Injured,Approach Overtaking Turn,Crash Time,Rd_Number,Rd_Section,City_Street
0,County Road,2020-01-01,0.0,0.0,0.0,0.0,0.0,CSP,On Roadway,Non-Intersection,...,NaN,Straight,Level,0.0,0.0,Not Applicable,00:00:00,NaN,SYCAM,NaN
1,City Street,2020-01-01,0.0,0.0,0.0,0.0,0.0,EGLSO,Ran off right side,Driveway Access Related,...,NaN,Curve Unknown Direction,Level,0.0,0.0,Not Applicable,10:00:00,NaN,NIGHT,NaN
2,State Highway,2020-01-01,1.0,1.0,0.0,0.0,0.0,CSP,On Roadway,Non-Intersection,...,ELK,Straight,Level,0.0,1.0,Not Applicable,10:00:00,160.0A,123.1,NaN
3,City Street,2020-01-01,0.0,0.0,0.0,0.0,0.0,ASO,On Roadway,At Intersection,...,NaN,Straight,Level,0.0,0.0,Not Applicable,60:00:00,NaN,JAMIS,NaN
4,City Street,2020-01-01,0.0,1.0,0.0,0.0,0.0,DPD,Ran off right side,Non-Intersection,...,NaN,Straight,Level,0.0,1.0,Not Applicable,13:00:00,NaN,AURAR,NaN


In [46]:
print(list(cdot_pdf_withNans_dedup.columns))

['System Code', 'Crash Date', 'Injury 00', 'Injury 01', 'Injury 02', 'Injury 03', 'Injury 04', 'Agency Id', 'Location', 'Road Description', 'Total Vehicles', 'Road Condition', 'Lighting Conditions', 'Weather Condition', 'First HE', 'Second HE', 'Third HE', 'MHE', 'Crash Type', 'Location 1', 'Link', 'Location 2', 'City', 'County', 'Latitude', 'Longitude', 'Crash Year', 'CUID', 'Fourth HE', 'Secondary Crash', 'Construction Zone', 'School Zone', 'Weather Condition 2', 'Lane Position', 'Record Status', 'Processing Status', 'Last Updated', 'TU-1 Special Function', 'TU-2 Special Function', 'TU-3 Special Function', 'TU-1 Autonomous Vehicle', 'TU-2 Autonomous Vehicle', 'TU-3 Autnomous Vehicle', 'TU-1 Speed', 'TU-2 Speed', 'TU-3 Speed', 'TU-1 Driver Action', 'TU-2 Driver Action', 'TU-3 Driver Action', 'TU-1 Safety System Available ', 'TU-2 Safety System Available ', 'TU-3 Safety System Avaliable ', 'TU-1 NM Facility Available', 'TU-2 NM Facility Available', 'TU-3 NM Facility Available', 'TU-1 N

In [47]:
# Select columns for analysis 
analysis_columns = [
    'Crash Date', 
    'City', 
    'County',
    'Crash Year',
    'Location 1', 
    'Location 2',
    'Link',
    'Latitude', 
    'Longitude',
]

cdot_pdf_select_columns = cdot_pdf_withNans_dedup[analysis_columns]

# check NaN count by column 
for col in cdot_pdf_select_columns.columns:
    print(col, cdot_pdf_select_columns.loc[cdot_pdf_select_columns[col].isna()].shape[0])

Crash Date 1
City 127263
County 1
Crash Year 1
Location 1 41
Location 2 340
Link 319
Latitude 49571
Longitude 49571


In [48]:
cdot_pdf_select_columns.describe()

,Crash Date,Crash Year,Latitude,Longitude
count,481614,481614.000000,432044.000000,432044.000000
mean,2022-07-28 13:07:26.294335488,2022.066236,38.474544,-102.330116
min,2020-01-01 00:00:00,2020.000000,0.000000,-10808.604000
25%,2021-05-18 00:00:00,2021.000000,39.253707,-105.094782
50%,2022-08-06 00:00:00,2022.000000,39.713000,-104.982800
75%,2023-10-23 00:00:00,2023.000000,39.867560,-104.821320
max,2024-12-31 00:00:00,2024.000000,3911.461000,0.000000
std,NaN,1.398973,8.788714,23.738573


In [49]:
# fill null values that screw up counting
cdot_pdf = cdot_pdf_select_columns.copy()
    
# Fill NaN values 
#cdot_pdf['Crash Date'] = cdot_pdf['Crash Date'].fillna('no crash date provided')
#cdot_pdf['Crash Year'] = cdot_pdf['Crash Year'].fillna('no crash date provided')
#cdot_pdf['City'] = cdot_pdf['City'].fillna('no city provided')
#cdot_pdf['County'] = cdot_pdf['County'].fillna('no county provided')
#cdot_pdf['Latitude'] = cdot_pdf['Latitude'].fillna(999.999000)
#cdot_pdf['Longitude'] = cdot_pdf['Longitude'].fillna(999.999000)

# mark "unknown" city, Location 1, Location 2 as null
cdot_pdf['City'] = cdot_pdf['City'].fillna('UNKNOWN')
cdot_pdf['Location 1'] = cdot_pdf['Location 1'].fillna('UNKNOWN')
cdot_pdf['Location 2'] = cdot_pdf['Location 1'].fillna('UNKNOWN')

cdot_pdf.loc[cdot_pdf['City'].str.contains('UNKNOWN'), 'City'] = None
cdot_pdf.loc[cdot_pdf['Location 1'].str.contains('UNKNOWN'), 'Location 1'] = None
cdot_pdf.loc[cdot_pdf['Location 2'].str.contains('UNKNOWN'), 'Location 2'] = None

In [50]:
# check NaN count by column 
for col in cdot_pdf.columns:
    print(col, cdot_pdf.loc[cdot_pdf[col].isna()].shape[0])

Crash Date 1
City 127265
County 1
Crash Year 1
Location 1 96
Location 2 96
Link 319
Latitude 49571
Longitude 49571


In [51]:
display(cdot_pdf.head(10))

,Crash Date,City,County,Crash Year,Location 1,Location 2,Link,Latitude,Longitude
0,2020-01-01,None,MESA,2020.0,SYCAMORE CT,SYCAMORE CT,00650FN,0.000000,0.000000
1,2020-01-01,GYPSUM,EAGLE,2020.0,NIGHTHAWK CIR,NIGHTHAWK CIR,AT,0.000000,0.000000
2,2020-01-01,None,ARCHULETA,2020.0,HWY 160,HWY 160,00528FE,37.204663,-107.329465
3,2020-01-01,CENTENNIAL,ARAPAHOE,2020.0,JAMISON DR 15700 E,JAMISON DR 15700 E,AT,39.578560,-104.807100
4,2020-01-01,DENVER,DENVER,2020.0,AURARIA PK,AURARIA PK,00050FW,39.746541,-105.007589
5,2020-01-01,THORNTON,ADAMS,2020.0,I-25,I-25,AT,40.135016,-104.980071
6,2020-01-01,DENVER,DENVER,2020.0,I-25 SB,I-25 SB,AT,39.706430,-104.996700
7,2020-01-01,None,WELD,2020.0,CR 15,CR 15,01056FS,40.011973,-104.923973
8,2020-01-01,PUEBLO,PUEBLO,2020.0,CROW CUTOFF,CROW CUTOFF,AT,0.000000,0.000000
9,2020-01-01,COLORADO SPRINGS,EL PASO,2020.0,VERDE DR,VERDE DR,00.50MW,38.813605,-104.774042


### Filter to El Paso County for simplicity 

In [52]:
cdot_pdf_elpaso = cdot_pdf.loc[cdot_pdf['County'] == 'EL PASO']
print(cdot_pdf_elpaso.shape)

# check NaN count by column 
for col in cdot_pdf_elpaso.columns:
    print(col, cdot_pdf_elpaso.loc[cdot_pdf_elpaso[col].isna()].shape[0])

(49417, 9)
Crash Date 0
City 10916
County 0
Crash Year 0
Location 1 4
Location 2 4
Link 12
Latitude 9073
Longitude 9073


### Attempt to Fill Missing Lat Long

In [53]:
cdot_elpaso_noLL = cdot_pdf_elpaso.loc[cdot_pdf_elpaso['Latitude'].isna() | cdot_pdf_elpaso['Longitude'].isna()]
display(cdot_elpaso_noLL)

,Crash Date,City,County,Crash Year,Location 1,Location 2,Link,Latitude,Longitude
27,2021-01-01,MONUMENT,EL PASO,2021.0,WINDING HILLS DR,WINDING HILLS DR,AT,NaN,NaN
33,2021-01-01,None,EL PASO,2021.0,GOLDFIELD DR,GOLDFIELD DR,AT,NaN,NaN
36,2021-01-01,None,EL PASO,2021.0,MARKSHEFFEL RD,MARKSHEFFEL RD,AT,NaN,NaN
151,2021-01-01,None,EL PASO,2021.0,TABLE ROCK RD,TABLE ROCK RD,AT,NaN,NaN
153,2021-01-01,None,EL PASO,2021.0,WOODMEN RD E,WOODMEN RD E,00500FW,NaN,NaN
...,...,...,...,...,...,...,...,...,...
33352,2024-04-30,MANITOU SPRINGS,EL PASO,2024.0,RUXTON AVE,RUXTON AVE,WO,NaN,NaN
45183,2024-06-14,None,EL PASO,2024.0,I-25,I-25,AT,NaN,NaN
48039,2024-06-26,None,EL PASO,2024.0,HWY 24,HWY 24,AT,NaN,NaN
72836,2024-09-25,MANITOU SPRINGS,EL PASO,2024.0,RUXTON AVE,RUXTON AVE,00128FW,NaN,NaN


In [54]:
# Write out to csv 
cdot_elpaso_noLL.to_csv('crashes_elpaso_w_no_latlong_2021-2024.csv', index=False) 

#### Filter to only entries with a number in either Location 1 or Location 2

In [59]:
cdot_elpaso_l1l2_num = cdot_elpaso_noLL.loc[cdot_elpaso_noLL['Location 1'].str.contains(r'\d') | cdot_elpaso_noLL['Location 2'].str.contains(r'\d')]
display(cdot_elpaso_l1l2_num)

cdot_elpaso_l1l2_num_notHWY = cdot_elpaso_l1l2_num.loc[~cdot_elpaso_l1l2_num['Location 1'].str.contains('I-25') & 
                                                       ~cdot_elpaso_l1l2_num['Location 1'].str.contains('HWY') & 
                                                       ~cdot_elpaso_l1l2_num['Location 1'].str.contains('INTERSTATE 25')]

display(cdot_elpaso_l1l2_num_notHWY)


,Crash Date,City,County,Crash Year,Location 1,Location 2,Link,Latitude,Longitude
1476,2021-01-07,COLORADO SPRINGS,EL PASO,2021.0,S 1500 BLOCK OF VERDE DR,S 1500 BLOCK OF VERDE DR,00100FW,NaN,NaN
1569,2021-01-08,COLORADO SPRINGS,EL PASO,2021.0,S 8TH ST,S 8TH ST,00200FS,NaN,NaN
2321,2021-01-11,COLORADO SPRINGS,EL PASO,2021.0,16500 RIDGELINE DR,16500 RIDGELINE DR,00575FW,NaN,NaN
3731,2021-01-17,COLORADO SPRINGS,EL PASO,2021.0,515 QUEBEC ST,515 QUEBEC ST,AT,NaN,NaN
4086,2021-01-19,None,EL PASO,2021.0,I-25 SB ENT RAMP,I-25 SB ENT RAMP,99999FS,NaN,NaN
...,...,...,...,...,...,...,...,...,...
99556,2023-12-21,COLORADO SPRINGS,EL PASO,2023.0,S 21ST ST,S 21ST ST,AT,NaN,NaN
5796,2024-01-20,COLORADO SPRINGS,EL PASO,2024.0,4990 WEAVER DR,4990 WEAVER DR,AT,NaN,NaN
15663,2024-02-23,MANITOU SPRINGS,EL PASO,2024.0,W HWY 24,W HWY 24,AT,NaN,NaN
45183,2024-06-14,None,EL PASO,2024.0,I-25,I-25,AT,NaN,NaN


,Crash Date,City,County,Crash Year,Location 1,Location 2,Link,Latitude,Longitude
1476,2021-01-07,COLORADO SPRINGS,EL PASO,2021.0,S 1500 BLOCK OF VERDE DR,S 1500 BLOCK OF VERDE DR,00100FW,NaN,NaN
1569,2021-01-08,COLORADO SPRINGS,EL PASO,2021.0,S 8TH ST,S 8TH ST,00200FS,NaN,NaN
2321,2021-01-11,COLORADO SPRINGS,EL PASO,2021.0,16500 RIDGELINE DR,16500 RIDGELINE DR,00575FW,NaN,NaN
3731,2021-01-17,COLORADO SPRINGS,EL PASO,2021.0,515 QUEBEC ST,515 QUEBEC ST,AT,NaN,NaN
4087,2021-01-19,COLORADO SPRINGS,EL PASO,2021.0,4800 MILTON E PKWY,4800 MILTON E PKWY,00030FE,NaN,NaN
...,...,...,...,...,...,...,...,...,...
92868,2023-11-29,COLORADO SPRINGS,EL PASO,2023.0,S 31ST ST,S 31ST ST,00140FN,NaN,NaN
97246,2023-12-13,COLORADO SPRINGS,EL PASO,2023.0,2300 VICKERS DR,2300 VICKERS DR,AT,NaN,NaN
98213,2023-12-16,COLORADO SPRINGS,EL PASO,2023.0,S 31ST ST,S 31ST ST,00062FE,NaN,NaN
99556,2023-12-21,COLORADO SPRINGS,EL PASO,2023.0,S 21ST ST,S 21ST ST,AT,NaN,NaN


In [61]:
# Write out to csv 
cdot_elpaso_l1l2_num_notHWY.to_csv('temp.csv', index=False) 